### Edit only first notebook cell, Define input output folders as your reqirements,Run next two cell to get output.

In [17]:
#path of the folder
# Edit the destination of input folder and output folder
image_folder = 'orginal_images/'
output_folder = 'blur_image/'
#jpg, png,jpeg images
# change according to your requirements
input_image_format='.jpg'
output_image_format='.jpg'
#define the shape of orginal images
width_output_images=2048
height_output_images=1288

In [ ]:
#!pip install pybboxes
#!pip install ultralytics
import os
import glob
import cv2
import pybboxes as pbx
from ultralytics import YOLO
det_model=YOLO("yolo.pt")#https://docs.ultralytics.com/models/yolov8/#performance-metrics"

In [18]:
det_model(source=image_folder,
            save=False,
            save_txt=True,
            conf=0.005,# normally set 0.1/ for more complex image set less than 0.1
            device='cpu',
            project="object",
            name="pred")
images = sorted(glob.glob(image_folder+"/*"))
os.mkdir("annot_txt")

text_dir=f'./object/pred/labels/'

try:
    for file in os.listdir(text_dir):
        if (file.endswith('.txt')):
            #frame_num = int(file.replace(".txt","").split("_")[1])
            with open(text_dir+file, 'r') as fin:
                for line in fin.readlines():
                    line = [float(item) for item in line.split()[1:]]
                    line = pbx.convert_bbox(line, from_type="yolo", to_type="voc", image_size=(width_output_images, height_output_images))
                    data_string = " ".join(str(num) for num in line)
                    with open(f"annot_txt/{os.path.basename(file)}", "a") as f:
                        f.write(data_string+"\n")
except:
    print(f'{os.path.basename(file)} has no detected objects.')
def anonymize(image, regions):
    """
    Blurs the image, given the x1,y1,x2,y2 cordinates using Gaussian Blur.
    """
    for region in regions:
        x1,y1,x2,y2 = region
        x1, y1, x2, y2 = round(x1), round(y1), round(x2), round(y2)
        roi = image[y1:y2, x1:x2]
        blurred_roi = cv2.GaussianBlur(roi, (31, 31), 0)
        image[y1:y2, x1:x2] = blurred_roi
    return image
txt_folder = 'annot_txt/'
# Create the output folder if it doesn't exist
#if not os.path.exists(output_folder):
#    os.makedirs(output_folder)

# List all text files in the 'dir' folder
txt_files = [f for f in os.listdir(txt_folder) if f.endswith('.txt')]

for txt_file in txt_files:
    # Read the text file containing bounding box information
    with open(os.path.join(txt_folder, txt_file), 'r') as f:
        lines = f.readlines()

    # Extract bounding box coordinates from the txt file
    bboxes = []
    for line in lines:
        values = line.strip().split()
        x_min, y_min, x_max, y_max = map(int, values)  # Assuming VOC format with x_min, y_min, x_max, y_max
        bboxes.append([x_min, y_min, x_max, y_max])

    # Read the corresponding image
    image_file = txt_file.replace('.txt', input_image_format)  # Assuming image files have .png extension
    image_path = os.path.join(image_folder, image_file)
    image = cv2.imread(image_path)

    # Apply Gaussian blur to each bounding box region
    for bbox in bboxes:
        image = anonymize(image, bboxes)

    # Save the blurred image to the output folder
    output_file = txt_file.replace('.txt', output_image_format)
    output_path = os.path.join(output_folder, output_file)
    cv2.imwrite(output_path, image)

print("Task Completed,HAppy coding:)")


image 1/1 c:\Users\Raikibul Hasan\Documents\GitHub\LF_Anonymization\orginal_images\image2.jpg: 704x1088 17 licenses, 6 faces, 2759.5ms
Speed: 17.0ms preprocess, 2759.5ms inference, 2.0ms postprocess per image at shape (1, 3, 704, 1088)
Results saved to object\pred
1 label saved to object\pred\labels
Task Completed,HAppy coding:)
